In [1]:
import tensorflow as tf
import torch

# CUDA 빌드 여부 확인 (TensorFlow 2.x)
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



2025-05-13 21:12:53.586780: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 21:12:53.601424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747138373.615316   18399 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747138373.619860   18399 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747138373.631939   18399 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Built with CUDA: True
Using device: cuda


In [ ]:
import huggingface_hub

huggingface_hub.login(token=

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "나에게 어떻게 하면 linux를 잘 사용할 수 있는지 알려줘"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


thinking content: <think>
Okay, the user wants to know how to use Linux properly. Let me start by breaking down the key steps they might need. First, understanding the basics is crucial. They should know how to open terminal, find the root user, and use basic commands like 'ls', 'cd', 'grep', etc. Then, moving to more advanced commands like 'grep' and 'find' is important. Also, setting up a password and managing files like 'passwd' and 'chown' would help. I should mention using 'sudo' for administrative tasks. Oh, and maybe some tips for security, like not using public folders. Let me structure this in a clear way, starting with the basics and moving to more advanced topics. Make sure to keep the language simple and conversational, avoiding technical jargon where possible.
</think>
content: Linux 사용을 잘 하기 위해서는 기초부터 익히고, 상위 기능을 배우는 것이 중요합니다. 아래 단계를 따라 하실 수 있습니다:

---

### **1. Linux 기초 이해**
- **Terminal 열기**:  
  `Ctrl + Alt + T` 또는 `sudo login`을 통해 terminal을 열어 시작하세요.  
  **기본 사용자(예: r